# Content-based Recommendations with PCA
## Similar movies have similar tags. How well is this similarity captured with PCA?

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1593049829309_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-2>

In [2]:
sc.install_pypi_package('pandas')
sc.install_pypi_package('scikit-learn')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl


In [3]:
df = sqlContext.read.csv('s3a://sparkdemonstration/movielens-tag-relevance.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import random
colsToShow = ['title'] + [random.choice(df.columns) for i in range(4)]
df.select(*colsToShow).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------------+--------------------+-------------------+
|               title|        intellectual|           quotable|         child abuse|interracial romance|
+--------------------+--------------------+-------------------+--------------------+-------------------+
|    Toy Story (1995)|             0.19075|            0.59925|0.012000000000000007|            0.08725|
|      Jumanji (1995)| 0.09200000000000004|0.09300000000000004|0.015500000000000012|            0.08725|
|Grumpier Old Men ...|0.059250000000000025|0.09899999999999998| 0.00974999999999998|0.14300000000000002|
|Waiting to Exhale...| 0.04949999999999999|0.08224999999999999| 0.01150000000000001|            0.41825|
|Father of the Bri...|               0.061|0.06724999999999998| 0.01150000000000001|            0.23475|
|         Heat (1995)|             0.34475|            0.50475| 0.02300000000000002|0.07350000000000001|
|      Sabrina (1995)|               0.064|0.0840000000

In [5]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
newCols = []

for c in df.columns:
    if "." in c:
        new_column = c.replace('.', '_')
        df = df.withColumnRenamed(c, new_column)
        newCols.append(new_column)
    else:
        newCols.append(c)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
assembler = VectorAssembler(inputCols=[c for c in newCols if c != 'title'],
                            outputCol='features')
scaler    = StandardScaler(inputCol='features', outputCol='normFeats', withMean=True)

df          = assembler.transform(df)
scalerModel = scaler.fit(df)
df          = scalerModel.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## PCA

In [8]:
rdd = df.select('normFeats').rdd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vectors

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
vectors = rdd.map(Vectors.dense)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
matrix = RowMatrix(vectors)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Get the PCs

In [12]:
pc = matrix.computePrincipalComponents(500)
matrix_reduced = matrix.multiply(pc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Nearest Neighbour Search in PC space

In [13]:
import numpy as np
X = matrix_reduced.rows.map(np.array).collect()
X = np.array(X)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
titles = df.select('title').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
import pandas as pd
pdf = pd.DataFrame(X, index=titles['title'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
pdf.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                          0          1    ...       498       499
title                                                     ...                    
Toy Story (1995)                     5.580545 -18.610977  ... -0.294096 -0.445163
Jumanji (1995)                     -11.108312 -13.988950  ... -0.500738  0.372158
Grumpier Old Men (1995)             -9.606716   1.298515  ... -0.127041  0.063340
Waiting to Exhale (1995)            -8.162652   6.345987  ... -0.533415 -0.344706
Father of the Bride Part II (1995) -10.634055   2.242504  ...  0.327135  0.084268

[5 rows x 500 columns]

In [17]:
from sklearn.neighbors import NearestNeighbors

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
n_pcs = 2
nn = NearestNeighbors()
nn = nn.fit(X[:, :n_pcs])
neighbors = nn.kneighbors(pdf.loc['Toy Story (1995)'].values[:n_pcs].reshape(1, -1), return_distance=False)
pdf.index[neighbors.ravel()].tolist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Toy Story (1995)', 'The Punisher: Dirty Laundry (2012)', "Empire of Dreams: The Story of the 'Star Wars' Trilogy (2004)", 'Ten Commandments, The (1956)', 'Blood of Heroes, The (Salute of the Jugger, The) (1989)']

<table>
    <tr>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/1/13/Toy_Story.jpg"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/c/cf/Poster_for_Dirty_Laundry.jpg"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/e/ef/Empire_of_Dreams.png"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/The_Ten_Commandments_%281956_film_poster%29.jpg/313px-The_Ten_Commandments_%281956_film_poster%29.jpg"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/2/2a/Bloodofheroes.jpg"></img></td>
        
</table>

## Increase the number of Principal Components

In [19]:
n_pcs = 10
nn = NearestNeighbors()
nn = nn.fit(X[:, :n_pcs])
neighbors = nn.kneighbors(pdf.loc['Toy Story (1995)'].values[:n_pcs].reshape(1, -1), return_distance=False)
pdf.index[neighbors.ravel()].tolist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Toy Story (1995)', 'Finding Nemo (2003)', 'Monsters, Inc. (2001)', "Bug's Life, A (1998)", 'Ratatouille (2007)']

<table>
    <tr>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/1/13/Toy_Story.jpg"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/2/29/Finding_Nemo.jpg"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/6/63/Monsters_Inc.JPG"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/c/cc/A_Bug%27s_Life.jpg"></img></td>
        <td><img src="https://upload.wikimedia.org/wikipedia/en/5/50/RatatouillePoster.jpg"></img></td>
        
</table>

In [20]:
n_pcs = 100
nn = NearestNeighbors()
nn = nn.fit(X[:, :n_pcs])
neighbors = nn.kneighbors(pdf.loc['Toy Story (1995)'].values[:n_pcs].reshape(1, -1), return_distance=False)
pdf.index[neighbors.ravel()].tolist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Toy Story (1995)', 'Monsters, Inc. (2001)', 'Toy Story 2 (1999)', "Bug's Life, A (1998)", 'Toy Story 3 (2010)']

In [21]:
n_pcs = 500
nn = NearestNeighbors()
nn = nn.fit(X[:, :n_pcs])
neighbors = nn.kneighbors(pdf.loc['Toy Story (1995)'].values[:n_pcs].reshape(1, -1), return_distance=False)
pdf.index[neighbors.ravel()].tolist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Toy Story (1995)', 'Toy Story 2 (1999)', 'Monsters, Inc. (2001)', 'Toy Story 3 (2010)', "Bug's Life, A (1998)"]

In [22]:
n_pcs = 10
nn = NearestNeighbors()
nn = nn.fit(X[:, :n_pcs])
neighbors = nn.kneighbors(pdf.loc['Conjuring, The (2013)'].values[:n_pcs].reshape(1, -1), return_distance=False)
pdf.index[neighbors.ravel()].tolist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Conjuring, The (2013)', 'The Conjuring 2 (2016)', 'Amityville Horror, The (1979)', 'The Autopsy of Jane Doe (2016)', 'Cujo (1983)']